In [322]:
import numpy as np
import sklearn
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn import svm
from sklearn.tree import DecisionTreeRegressor

import matplotlib.pyplot as plt
import seaborn as sns
from pandas.plotting import scatter_matrix
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.base import BaseEstimator , TransformerMixin
from sklearn import preprocessing
from sklearn import utils
from sklearn.linear_model import LinearRegression
import joblib
from sklearn.model_selection import GridSearchCV 

In [376]:
x_train = pd.read_csv(r'C:\Users\avoav\Introduction to Data Science\Project סמסטר א\project-semester-a\ScaledTrainSet_data.csv' , index_col  = 'index')
x_train

,year,is_GAC,is_maserati,is_daihatsu,is_porsche,is_ds,is_smart,is_seres,is_cadillac,is_דודג',...,is_leasing,is_company,is_private,next_test_in,annual_licensing_fee,car_length,large,small,mini,medium
index,,,,,,,,,,,,,,,,,,,,,
1608,0.952381,0,0,0,0,0,0,0,0,0,...,0,0,1,0.000000,0.645547,0.764326,1,0,0,0
1329,0.761905,0,0,0,0,0,0,0,0,0,...,0,0,1,0.166667,0.188003,0.600759,0,0,0,1
1782,0.333333,0,0,0,0,0,0,0,0,0,...,0,0,1,0.166667,0.050377,0.652751,0,0,0,1
1193,0.523810,0,0,0,0,0,0,0,0,0,...,1,0,0,0.583333,0.027266,0.493359,0,1,0,0
1166,1.000000,0,0,0,0,0,0,0,0,0,...,0,0,1,0.833333,1.000000,0.489943,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1650,0.809524,0,0,0,0,0,0,0,0,0,...,0,0,1,0.333333,0.122825,0.518027,0,1,0,0
1100,0.428571,0,0,0,0,0,0,0,0,0,...,0,0,1,0.416667,0.142041,0.813283,1,0,0,0
1135,0.714286,0,0,0,0,0,0,0,0,0,...,0,0,1,0.000000,0.295767,0.675522,0,0,0,1


In [524]:
x_target = pd.read_csv(r'C:\Users\avoav\Introduction to Data Science\Project סמסטר א\project-semester-a\TrainSet-target_data.csv' , index_col='index' )


In [538]:
#  ע"מ לנרמל גם את עמודת המחירים של הטריין השתמשנו בפונקציה הזו 

lab_enc = preprocessing.LabelEncoder()
x_target = lab_enc.fit_transform(np.ravel(x_target, order = 'C'))
#print(utils.multiclass.type_of_target(x_target)) 

# או זו:

min_max_scaler = MinMaxScaler()
scaledtarget = x_target.copy()
scaledtarget[["price"]] = min_max_scaler.fit_transform(scaledtarget[["price"]])

# Cross-validation :

first we return the demi-model (the average) and try to be better than him.... 

In [539]:
 # בדיקת מודל אפס: 

def targetTransform(len_target):
    target_ = np.zeros(len_target)
    for i in range (len_target):
        target_[i] +=  x_target.mean()
    return target_

x_train1 , x_test1 , y_train1 , y_test1= sklearn.model_selection.train_test_split(x_train, x_target , test_size=0.1 , random_state = 2) 

pred = targetTransform(len(y_test1))
#print (f"\n{pred}")
print (f"RMSE: {np.sqrt(mean_squared_error(y_test1 , pred))}")

#במידה והמודל היה מחזיר תמיד את ממוצע מחיר הרכבים בטבלה זו הייתה מידת הדיוק שלו
# מדוייק יותר- כרגע הגענו למרחק הממוצע מחיזוי נכון בשקלים.

#29915.4789509615 = 0.8733269871229924
# 77634.8801434407 = פי 2.5 פחות טוב = 0.3853355463
print ("accuracy :" , 0.3853355463)

RMSE: 77634.8801434407
accuracy : 0.3853355463


In [540]:
#Dummy-model method:
    #Checking the prices mean, and then its percent.
    #Count the prices that between (price+percent) and (price-percent), duo the price is a spesific number (10+ digits)
    #Divide those counted prices by the amount of prices.

x_target.mean() # 0.19357003724451638
x_target.mean()/100 # 0.0019357003724451637

maxLim = x_target.mean() + x_target.mean()/100
minLim = x_target.mean() - x_target.mean()/100
counter = x_target.where(x_target > minLim).where(x_target < maxLim).dropna().count()
print ("accuracy:" , (float)(counter / x_target.count()))
#וזה אחוז הדיוק שאנחנו צריכים להיות מעליו

accuracy: 0.015706806282722512


אפשר לראות שאם נשתמש ב"מודל אפס" שמחזיר פשוט את הממוצע של המחירים בטבלה
הטעות חמורה בהרבה מאשר במודל שלנו 

## select and train a model

# Linear Regression model

In [541]:
#את הדאטה שהקדשנו לטריין אנחנו מפצלים שוב ע"מ לבחון את מודל הרגרסיה הלינארית עם רנדום אחר

x_train1 , x_test1 , y_train1 , y_test1= sklearn.model_selection.train_test_split(x_train, np.ravel(x_target, order = 'C') , test_size=0.2 , random_state = 2) 

lin_reg =  LinearRegression()
fit1= lin_reg.fit(x_train1, y_train1)

pred = fit1.predict(x_test1)
#print (f"\n{pred}")
print (f"RMSA: {np.sqrt(mean_squared_error(y_test1 , pred))}\n")

print ("score:" , lin_reg.score( x_test1, y_test1))


RMSA: 29915.4789509615

score: 0.8733269871229924



לפני כן הדיוק היה די גרוע, ניסינו להבין למה לא הצלחנו לחזות טוב עם המודל שלנו... עברנו על המכוניות ידנית וראינו באילו מכוניות הטעויות הגדולות קרו
החלטנו למחוק את הדוגמאות הקיצוניות וקיבלנו שיפור עצום

In [545]:
def display_scores(scores):
    print ("scores:",  scores)
    print("Mean:" , scores.mean() )
    print ("Standard deviation:" , scores.std()) 
    
# x_train4 , x_test4 , y_train4 , y_test4= sklearn.model_selection.train_test_split(x_train, x_target , test_size=0.2 , random_state = 4)   

# lin_reg =  LinearRegression()
# lin_reg = lin_reg.fit(x_train4, y_train4)

scores = cross_val_score(lin_reg, x_train , x_target ,cv=40 , scoring = "r2" )

display_scores(scores)
print ("score:" , lin_reg.score(x_test4 , y_test4))


scores: [ 8.43414762e-01  8.71194599e-01  8.65839748e-01  9.16968917e-01
  8.72831535e-01  8.47198421e-01  8.89466427e-01  8.94642078e-01
  8.49158091e-01  8.88292806e-01  9.15643351e-01  9.11263524e-01
  9.15432036e-01  8.33701628e-01  9.01462248e-01  8.26081489e-01
  8.83694055e-01  8.49532422e-01  8.93541397e-01  8.84106372e-01
  8.37016749e-01  9.16553636e-01  7.42212229e-01  8.55401282e-01
  8.50988355e-01  8.10184164e-01  9.21055808e-01  8.87566675e-01
  9.32249068e-01  9.37454449e-01  4.82220913e-01  7.65261854e-01
  7.29377139e-01 -3.29555327e+20 -2.14289428e+23  9.07658082e-01
  8.89855007e-01 -2.76625020e+18  9.03979299e-01  9.32477753e-01]
Mean: -5.365543732887243e+21
Standard deviation: 3.34546353562402e+22
score: -3.616475975610805e+20


In [543]:
#This method check which cv gives the best score.
    #if we make this one a function -> the function will return the bestCV
CV_COUNT = 100
maxScore = 0
bestCV = 0
bestScoreI = 0
scr = np.zeros(CV_COUNT)

for i in np.arange(2, CV_COUNT+2):
    bestScoreI = 0
    ind = cross_val_score(lin_reg, x_train, x_target, cv=i, scoring = "r2")
    for j in ind:
        if j > maxScore:
            maxScore = j
            bestCV = i
        if j > bestScoreI:
            scr[i-2] = pow(j, 2)    
    
print("r =", maxScore)
print("r^2 =", pow(maxScore, 2))
print("for cv =", bestCV)
#print(scr)
#print (cross_val_score(lin_reg, x_train, x_target, cv=14)) #a method to print scores by cv.

r = 0.9740470477033635
r^2 = 0.9487676511396385
for cv = 77


In [551]:
m = 0
whatCV = 0

for i in np.arange(2, 40):
    ind = cross_val_score(lin_reg, x_train, x_target, cv=i)
    for j in ind:
        if j > m:
            m = j
            whatCV = i
    
print(m)
print("for cv=", i)
# cv=39 זהו ההיפר משתנה הכי טוב שלנו ברגרסיה ליניארית

0.9545137277864899
for cv= 39


# DecisionTreeRegressor model

In [517]:
x_train1 , x_test1 , y_train1 , y_test1= sklearn.model_selection.train_test_split(x_train, x_target , test_size=0.1 , random_state = 2) 


tree_reg =  DecisionTreeRegressor()
fit1 = tree_reg.fit(x_train1 , y_train1)
pred = fit1.predict(x_test1)
#print (f"\n{pred}")
print (f"RMSA: {np.sqrt(mean_squared_error(y_test1 , pred))}\n")
print ("score:" , tree_reg.score(x_test1 , y_test1))

RMSA: 35414.93098683433

score: 0.7904761469770469


#  gridsearch for Linear Regression:


In [509]:

param_grid_LR = [  {'C' : [1 ,3 ,5 ] ,  'kernel': ['linear' , 'rbf']} ]

grid_search = GridSearchCV(svm.SVC(gamma = 'auto') , param_grid_LR , cv = 3,
                                   scoring = 'neg_mean_squared_error' , return_train_score = False)
grid_search.fit(x_train1, y_train1)
df = pd.DataFrame(grid_search.cv_results_)
df[['param_C' , 'param_kernel' , 'mean_test_score']]



C:\Users\avoav\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(("The least populated class in y has only %d"


,param_C,param_kernel,mean_test_score
0,1,linear,-3.093177e+09
1,1,rbf,-6.158225e+09
2,3,linear,-3.188704e+09
3,3,rbf,-5.887711e+09
4,5,linear,-2.910721e+09
5,5,rbf,-5.773339e+09


In [502]:
print ("best score :" , grid_search.best_score_)
print ("best params:" , grid_search.best_params_)


best score : -2910721104.1374474
best params: {'C': 5, 'kernel': 'linear'}


In [550]:
# שמירה של המודל: 
LR_model =  LinearRegression()
LR_model.fit(x_train1 , y_train1)
joblib.dump(LR_model , 'Linear Regression-model_1.joblib')

#שימוש במודל קיים : 
model = joblib.load('Linear Regression-model_1.joblib')
pred = model.predict(x_test1)
model.score(x_test1 , y_test1)

0.8733269871229924

#  gridsearch for tree Regressor:


In [511]:
param_grid_DT = [{'splitter' : ['best' , "random"] , 'max_features':[10, 15 ,20 , 40] , 'min_samples_split': [2 , 3 , 5, 15], }
                ]
grid_search = GridSearchCV(tree_reg , param_grid_DT , cv = 3,
                                   scoring = 'neg_mean_squared_error' , return_train_score = False)
grid_search.fit(x_train1, y_train1)
df = pd.DataFrame(grid_search.cv_results_)

df[['param_max_features' , 'param_min_samples_split' , 'param_splitter' , 'params' , 'mean_test_score']]

print ("best score :" , grid_search.best_score_)
print ("best params:" , grid_search.best_params_)


best score : -902201774.6013699
best params: {'max_features': 40, 'min_samples_split': 15, 'splitter': 'best'}


In [549]:
# שמירה של המודל: 
tree_model =  DecisionTreeRegressor()
tree_model.fit(x_train1 , y_train1)
joblib.dump(tree_model , 'tree-model_1.joblib')

#שימוש במודל קיים : 
model = joblib.load('tree-model_1.joblib')
pred = model.predict(x_test1)
model.score(x_test1 , y_test1)

0.8148117765661046

## found the best classifier and hiperparameters , for the two models

In [554]:
X_train, X_test, y_train, y_test = train_test_split(x_train,np.ravel(x_target, order = 'C') , test_size=0.20, random_state=4)

names = ["Decision Tree", "Linear Regression", "zero model"]

param_grid_DT = [{'splitter' : ['best' , "random"] , 'max_features':[2 , 10 ,20] , 'min_samples_split': [2 , 3 , 5]}
                ]
param_grid_LR =  [  {'C' : [1 ,3 ,5] ,  'kernel': ['linear' , 'rbf']} ]

classifiers = [
    DecisionTreeRegressor()
   , LinearRegression() ]

score = 0
for name, clf in zip(names, classifiers):
    if name == "Decision Tree":
        clf = DecisionTreeRegressor()
        grid_search = GridSearchCV(clf, param_grid=param_grid_DT)
        grid_search.fit(X_train, y_train)
        if grid_search.best_score_ > score:
            score = grid_search.best_score_
            best_clf = grid_search
    
    elif name == "Linear Regression":
        clf = svm.SVC(gamma = 'auto')
        grid_search = GridSearchCV(svm.SVC(gamma = 'auto'), param_grid_LR , cv = 3,
                                   scoring = 'neg_mean_squared_error' , return_train_score = True)
        grid_search.fit(X_train, y_train)
        if grid_search.best_score_ > score:
            score = grid_search.best_score_
            best_clf = grid_search


print("best classifier: ", best_clf, " Accuracy= ", score)


C:\Users\avoav\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(("The least populated class in y has only %d"


best classifier:  GridSearchCV(estimator=DecisionTreeRegressor(),
             param_grid=[{'max_features': [2, 10, 20],
                          'min_samples_split': [2, 3, 5],
                          'splitter': ['best', 'random']}])  Accuracy=  0.8107050958564695


In [520]:
#trying....
clf = svm.SVC(kernel='linear', C=1, random_state=2).fit(x_train ,np.ravel(x_target, order = 'C') )
scores = cross_val_score(clf, x_train , x_target, cv=3)
scores


C:\Users\avoav\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(("The least populated class in y has only %d"


array([0.03475336, 0.02693603, 0.03030303])

In [521]:
#so that is the best score :  
clf = svm.SVC(kernel='linear', C=5, random_state=2).fit(x_train ,np.ravel(x_target, order = 'C') )
scores = cross_val_score(clf, x_train , x_target, cv=3)
scores[0]

C:\Users\avoav\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(("The least populated class in y has only %d"


0.026905829596412557